This is a ResNet-based CNN for binary image classification (Cat vs Dog)

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
# Use the ImageDataGenerator from the tensorflow runtime to avoid static import resolution issues
ImageDataGenerator = tf.keras.preprocessing.image.ImageDataGenerator
# Access keras submodules via the tf object to avoid static import resolution issues
layers = tf.keras.layers
models = tf.keras.models
Model = tf.keras.Model
import numpy as np

In [2]:
train_dir = "./train"  
test_dir = "./test"   

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (224, 224)  # ResNet requires 224x224 input
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 3788 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 945 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [6]:
# RESNET CNN MODEL ARCHITECTURE
# Transfer learning with ResNet50 pre-trained on ImageNet

# Load pre-trained ResNet50 without top layers
base_model = tf.keras.applications.ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)
)

# Freeze the base model initially
base_model.trainable = False

# Build custom top layers with regularization and dropout
inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)

# Create the complete model
model = Model(inputs=inputs, outputs=outputs)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 84s 1us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 84s 1us/step


In [7]:
# Display model architecture summary
print("[INFO] Model Architecture Summary:")
model.summary()

[INFO] Model Architecture Summary:


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 2048)           │         8,192 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,153,473 (92.14 MB)

 Trainable params: 561,665 (2.14 MB)

 Non-trainable params: 23,591,808 (90.00 MB)

In [8]:
# Configure the model optimizer, loss function, and metrics
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [9]:
# TRAINING THE CNN - PHASE 1: FROZEN BASE MODEL
history_phase1 = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 105s 849ms/step - accuracy: 0.6993 - loss: 0.5879 - val_accuracy: 0.5016 - val_loss: 0.7609
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 105s 849ms/step - accuracy: 0.6993 - loss: 0.5879 - val_accuracy: 0.5016 - val_loss: 0.7609
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 103s 864ms/step - accuracy: 0.7474 - loss: 0.5317 - val_accuracy: 0.6931 - val_loss: 0.5720
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 103s 864ms/step - accuracy: 0.7474 - loss: 0.5317 - val_accuracy: 0.6931 - val_loss: 0.5720
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 101s 850ms/step - accuracy: 0.7629 - loss: 0.5109 - val_accuracy: 0.8180 - val_loss: 0.4867
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 101s 850ms/step - accuracy: 0.7629 - loss: 0.5109 - val_accuracy: 0.8180 - val_loss: 0.4867
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 111s 936ms/step - accuracy: 0.7706 - loss: 0.4967 - val_accuracy: 0.8148 - val_loss: 0.4428
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 111s 936ms/step - accuracy: 0.7706 -

In [10]:
# FINE-TUNING - PHASE 2: UNFREEZE BASE MODEL
# Unfreeze the base model
base_model.trainable = True

# Freeze the first 100 layers to retain low-level features
for layer in base_model.layers[:100]:
    layer.trainable = False

# Recompile with a lower learning rate for fine-tuning
fine_tune_lr = 0.0001 / 10
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=fine_tune_lr),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Continue training with unfrozen layers
history_phase2 = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.6972 - loss: 0.7455 - val_accuracy: 0.6603 - val_loss: 0.6148
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 190s 2s/step - accuracy: 0.6972 - loss: 0.7455 - val_accuracy: 0.6603 - val_loss: 0.6148
Epoch 2/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.7186 - loss: 0.6722 - val_accuracy: 0.7249 - val_loss: 0.5442
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.7186 - loss: 0.6722 - val_accuracy: 0.7249 - val_loss: 0.5442
Epoch 3/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7355 - loss: 0.6343 - val_accuracy: 0.8032 - val_loss: 0.4231
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 180s 2s/step - accuracy: 0.7355 - loss: 0.6343 - val_accuracy: 0.8032 - val_loss: 0.4231
Epoch 4/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.7489 - loss: 0.6035 - val_accuracy: 0.8180 - val_loss: 0.3955
Epoch 5/10
119/119 ━━━━━━━━━━━━━━━━━━━━ 194s 2s/step - accuracy: 0.7489 - loss: 0.6035 - val_accu

In [11]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 27s 726ms/step - accuracy: 0.8269 - loss: 0.3914
Test Accuracy: 0.8268581032752991
37/37 ━━━━━━━━━━━━━━━━━━━━ 27s 726ms/step - accuracy: 0.8269 - loss: 0.3914
Test Accuracy: 0.8268581032752991


In [12]:
# SAVE THE MODEL
model.save('exercise_6_resnet_ledesma.h5')

In [32]:
# This is important to map the predicted class indices back to class labels
print(train_generator.class_indices)

{'chihuahua': 0, 'muffin': 1}


In [33]:
# SIMPLE INFERENCE SCRIPT
# Use the tensorflow runtime object 'tf' already imported in the notebook
# to access the preprocessing utilities to avoid static import resolution issues.
image = tf.keras.preprocessing.image

def predict_image(img_path, model_path='exercise_6_resnet_ledesma.h5'):
    model = tf.keras.models.load_model(model_path, compile=False)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]    

    # Since train_generator.class_indices is {'chihuahua': 0, 'muffin': 1}
    # my label layout should be:
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"Prediction: {label} (confidence: {pred:.2f})")

In [34]:
# Example usage:
predict_image("./predict/predict_1.jpg")
predict_image("./predict/predict_2.jpg")
predict_image("./predict/predict_3.jpg")
predict_image("./predict/predict_4.jpeg")
predict_image("./predict/predict_5.webp")
predict_image("./predict/predict_6.webp")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 930ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 930ms/step
Prediction: Muffin (confidence: 0.99)
Prediction: Muffin (confidence: 0.99)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 914ms/step
Prediction: Muffin (confidence: 0.98)
Prediction: Muffin (confidence: 0.98)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 933ms/step
Prediction: Muffin (confidence: 0.97)
Prediction: Muffin (confidence: 0.97)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 909ms/step
Prediction: Chihuahua (confidence: 0.39)
Prediction: Chihuahua (confidence: 0.39)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 968ms/step
Prediction: Chihuahua (confidence: 0.36)
Prediction: Chihuahua (confidence: 0.36)
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 927ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 927ms/step
Prediction: Chihuahua (confidence: 0.02)
Prediction: Chihuahua (confidence: 0.02)
